In [1]:
import time
import gurobipy as gp
from gurobipy import *
import xlrd
import numpy as np
import tsp
import pandas as pd
from statistics import mean, stdev
from operator import truediv
from python_tsp.exact import solve_tsp_dynamic_programming
import json

# data import
M = []
sheet = pd.read_csv(r'C:\Users\dell\Desktop\IIT Slides\sem7\BTP\xyz.csv',header=None)
N = len(sheet)
b = 6  # expected shopping time at a stock point

for i in range(0, N):
    row = []
    for j in range(0, N):
        row.append(sheet.iloc[i][j])
    M.append(row)
C = M.copy()
# passage = [1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 16, 18, 20, 23, 25, 27, 30, 31, 32, 33, 34, 37, 39, 41, 44,
#            46, 48, 51, 53, 55, 58, 59, 60, 61, 62]
# master_list = passage.copy()
# master_list.remove(1)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\dell\\Desktop\\IIT Slides\\sem7\\BTP\\xyz.csv'

In [ ]:
def shortest(c,n, v):
    r = range(len(c))
    permutation, distance = solve_tsp_dynamic_programming(c);
    obj = distance
    t = n*b + 5*obj/v
    return t

In [192]:
Out = pd.DataFrame(columns =['arrival rate','probabilty','itr', 'num_customers',  'contacts', 'mean(run_time)', 'stdev(run_time)'])

In [193]:
pip install python-tsp

Note: you may need to restart the kernel to use updated packages.


In [194]:


def compute_path(n, d, dx, y, t0): # n = numbr of nodes, d(matrix),d[i,j]-direct route time from i to j,
    #dx(matrix),dx[i,k] = time when ith node is visited by kth agent, where k<K, t0 = entry time of the agent
    n = n-1 # to exclude n+1 th node from array(to compensate)
    M = 100000 # big_M
    nodes = range(0, n) # to excude destination node (0,1,2-----,n-1)
    nodes2 = range(1, n+1) # to exclude source node (1,2,3,------n)
    nodes3 = range(1,n) # to exclude d and s 
    nodes4 = range(0,n+1) # include all vertex

    maxlen = 0
    for i in range(0, n+1):
        if len(dx[i]) > maxlen:
            maxlen = len(dx[i]) # to find max number of agent visiting any node

    for i in range(0, n+1): # append 0 till the maxlen is reached fro every node (5,3,7,8)
        while len(dx[i]) < maxlen:                                              #(4,2,0,0)
            dx[i].append(0)                                                     #(3,0,0,0) 

    e = range(0, maxlen) #(0,1,--maxlen)
    start_time = time.time()

    with gp.Env(empty=True) as env: # setting environment for gurobi optimizer
        env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model(env=env) as m1: # giving a model name m1

            # VARIABLES
            
            t = m1.addVars(n+1, lb=0, name="t") # time of arrival at each node
            u = m1.addVars(n+1, lb=1, ub=n+1, vtype=GRB.INTEGER, name="u") # order of visiting of nodes (1,3,5,4,2,6) 2nd node is visited 3rd
            x = m1.addVars(n+1, n+1, vtype=GRB.BINARY, name="x") # if i comes before j , x[i][j]=1 else 0
            y = m1.addVars(n+1, maxlen , vtype=GRB.BINARY, name="y") # if kth agent visits ith node after current agent; y[i,k]=1 else 0
            p = m1.addVars(n+1, maxlen, lb=0, name="p")
            z = m1.addVars(n+1, maxlen, lb=-gp.GRB.INFINITY, ub=gp.GRB.INFINITY, name="z")
            delta = m1.addVars(n+1, maxlen, vtype=GRB.BINARY, name="delta") # 1 if collision at ith node between kth agent and new agent, else 0
    
            obj = t[n]-t[0] # objective function for time 
        
            # CONSTRAINTS
            
            m1.addConstr(t[0] == t0)
            m1.addConstr(u[n] == n+1) # we have to return back to the initial node
            m1.addConstr(u[0] == 1) # start from 0
           
                        
                    
            for i in nodes: #(0 to n-1)
                for j in nodes2: #(1 to n)
                    if i!=j: # toavoid the constraint to be applied to i==j
                        m1.addConstr(x[i,j] + x[j,i] == 1)
                        m1.addConstr((u[j] - u[i]) >= x[i,j] - n*(1-x[i,j])) # to include order constraint
            for i in nodes2:# (1,2,3,...,n)
                m1.addConstr(x[0,i]==1)
                m1.addConstr(x[n,i]==0)
                m1.addConstr(x[i,i]==0)
            
            for k in e:
                m1.addConstr(delta[0, k] == 0)
                m1.addConstr(delta[n, k] == 0)
            for i in nodes2: # time constraint 
                for j in nodes2:
                    if i!=j:
                        m1.addConstr((t[j]-t[i]) >= (d[i][j]+b)*(x[i,j]) - M*(1-x[i,j]))
            for j in nodes2:
                m1.addConstr((t[j]-t[0]) >= d[0][j])
            
            for i in nodes2: # only for nodes(1,2,3,4...n-1) and not 0,n
                for k in e:
                    
                    if (dx[i][k]!=0):
                        m1.addConstr(t[i] + b <= dx[i][k]*y[i,k] + M*(1-y[i,k]) ) # if t[i] = 3,dx[i][k] = 4, b = 2, -1<z<1 contact will take place
                        m1.addConstr( t[i] - b >= dx[i][k]*(1-y[i,k]) - M*y[i,k] ) #TSP NC condition
                        
                        m1.addConstr(z[i, k] == (t[i] - dx[i][k]) / b) # if t[i] = 3,dx[i][k] = 4, b = 2, -1<z<1 contact will take place
                        m1.addConstr(p[i, k] == abs_(z[i, k])) # if p>=1 then no contact else contact 
                        m1.addConstr(M * delta[i, k] + p[i, k] >= 1) # if p[i,k]>1 redundant
                        m1.addConstr(-M * (1 - delta[i, k]) + p[i, k] <= 1) # if p[i,k]<=1 redundant
                    else:
                        m1.addConstr(delta[i, k] == 0)
                    
            
            
            # OPTIMIZATION

            m1.setObjective(obj, GRB.MINIMIZE)
            m1.optimize()
            end_time = time.time()
            h2 = m1.getVars()
            order = []
            t = []
            contacts = 0
           
            for item2 in delta.values():
                contacts += item2.x
                
                
                        
          
        
            for i in range(0, 2*(n+1)):
                if i < n+1:
                    t.append(np.round(h2[i].x, 1))
                else:
                    order.append(h2[i].x)
            output = [order, t, contacts,float(end_time-start_time),t[n]-t[0]]
            return output


# main loop
for length in range(100,1000,50):
    l = length/60

    max_ticks = 180  # run simulation for 10 hours, first 2 hours is warmup period

    for itr in range(0, 2):
        num_customers = 0
        bad_customers = 0
        contacts = 0
        times = []
        last = 0
        arrival_times = []
        shopping_times = []
        list_size = []
        speed = []
        time_added = []
        run_time = []
        list_occupied_nodes = []
        Y = np.zeros(N)
        customer_data = []
        for k in range(0, N):
            empty = list([])
            list_occupied_nodes.append(empty)

        for i in range(0, N):
            if len(list_occupied_nodes[i]) == 0:
                Y[i] = 0
                list_occupied_nodes[i].append(0)
            else:
                Y[i] = 1

        for i in range(0, 50):
            r = np.random.uniform(0, 1, 1)
            np.random.seed(0)
            r = np.random.exponential(1/l,1)[0]
            #inter = -np.log(1-r)/l # why cummulative 
            t = last + r
            t_prime = np.round(t, 1)
            times.append(float(t_prime))
            last = t_prime

        for q in times:
            if q < max_ticks:
                my_list = [0]
                v = 10 * np.random.randint(1, 3, 1)
                arrival_times.append(float(q))
                num_customers += 1

                # create list of customer
                for i in range(1,N-1): # available nodes
                    p = np.random.uniform(0, 1, 1) # probaility that a node is visited
                    if p < 0.2: 
                        my_list.append(i) # selecting the rows with probablity 0.2
                
                if(len(my_list)>10):
                    my_list = my_list[:10]
                    
                
                
                list_size.append(len(my_list))

                c = np.array(C)
                d = list_occupied_nodes.copy()
                y = Y.copy()
                my_list.append(N-1)
                size = len(my_list)
                for i in range(0, N):
                    if not (N - i - 1) in my_list:
                        c = np.delete(c, N - i - 1, 0) # deleting the row of node not to be visited(not in the list)
                        c = np.delete(c, N - i - 1, 1) # deleting the column of node not to be visited(not in the list)
                        d.pop(N - i - 1)
                        y = np.delete(y, N - i - 1, None)
                #print(size)
                #print(c.shape)
                #size = size-1
                result = compute_path(size, c, d, y, q)
                zip1 = zip(result[0], my_list)
                zip2 = zip(result[0], result[1])
                temp1 = [x for _, x in sorted(zip1)]
                temp2 = [x for _, x in sorted(zip2)]
                temp_dict = {"time": q, "travel_order": temp1, "timestamp": temp2}
                print(temp1)#order of visit
                print(temp2)#times
                print(result[2])
                print(result[4])
                customer_data.append(temp_dict)

                #if q > 60:
                end_time = temp2[size - 1] + b + 5 * M[0][temp1[size - 1]] / v
                arrival_times.append(q)
                s = np.round(end_time - q, 1)
                shopping_times.append(float(s))
                speed.append(float(v))
                run_time.append(result[3])
                list_size.append(size)
#                 alt = shortest(c, size, v)
#                 saved = np.round(s - alt, 1)
#                 time_added.append(float(saved))
                if result[2] > 0:
                    bad_customers += 1
                    contacts += 2*result[2]

                new_blockages = result[1]
                for i in range(1, size):
                    index = my_list[i]
                    list_occupied_nodes[index].append(new_blockages[i])

                for z in range(0, N):
                    for item in list_occupied_nodes[z]:
                        if item + b < q:
                            list_occupied_nodes[z].remove(item)
                for z in range(0, N):
                    if len(list_occupied_nodes[z]) == 0:
                        list_occupied_nodes[z].append(0)

                # print(list_occupied_nodes)
                for z in range(0, N):
                    if list_occupied_nodes[z][0] == 0:
                        Y[z] = 0
                    else:
                        Y[z] = 1

            else:# break when arrival time exceeds max_ticks
                break

        print("itr " + str(itr + 1))
        print(num_customers)

        #normalized_shop_t = list(map(truediv, shopping_times, list_size))
        #normalized_t_added = list(map(truediv, time_added, list_size))
        #norm_speed1 = []
        #norm_speed2 = []
        #norm_t_added1 = []
        #norm_t_added2 = []
#         for i in range(0, len(speed)):
#             if speed[i] == 10:
#                 norm_speed1.append(float(normalized_shop_t[i]))
#                 norm_t_added1.append(float(normalized_t_added[i]))
#             else:
#                 norm_speed2.append(float(normalized_shop_t[i]))
#                 norm_t_added2.append(float(normalized_t_added[i]))

        row = [l,p,itr + 1, num_customers, contacts, mean(run_time), stdev(run_time)]
        Out.loc[len(Out)] = row
        print(row)


# with open('tspmc55.txt', 'w') as outfile:
#     json.dump(Out, outfile)



[0, 7, 9]
[0.5, 2.5, 9.5]
0.0
9.0
[0, 5, 7, 6, 9]
[1.0, 2.0, 9.0, 16.0, 23.0]
0.0
22.0
[0, 1, 9]
[1.5, 2.5, 15.5]
0.0
14.0
[0, 8, 9]
[2.0, 4.0, 29.0]
0.0
27.0
[0, 8, 1, 9]
[2.5, 10.0, 18.0, 35.0]
0.0
32.5
[0, 4, 9]
[3.0, 4.0, 41.0]
0.0
38.0
[0, 9]
[3.5, 47.0]
0.0
43.5
[0, 7, 4, 3, 1, 9]
[4.0, 15.0, 22.0, 29.0, 36.0, 53.0]
0.0
49.0
[0, 6, 4, 9]
[4.5, 22.0, 29.0, 59.0]
0.0
54.5
[0, 5, 2, 9]
[5.0, 8.0, 15.0, 65.0]
0.0
60.0
[0, 5, 3, 9]
[5.5, 14.0, 35.0, 71.0]
0.0
65.5
[0, 5, 3, 1, 9]
[6.0, 20.0, 41.0, 48.0, 77.0]
0.0
71.0
[0, 9]
[6.5, 83.0]
0.0
76.5
[0, 9]
[7.0, 89.0]
0.0
82.0
[0, 9]
[7.5, 95.0]
0.0
87.5
[0, 4, 9]
[8.0, 35.0, 101.0]
0.0
93.0
[0, 5, 4, 9]
[8.5, 26.0, 41.0, 107.0]
0.0
98.5
[0, 7, 5, 1, 9]
[9.0, 21.0, 32.0, 54.0, 113.0]
0.0
104.0
[0, 5, 4, 9]
[9.5, 38.0, 47.0, 119.0]
0.0
109.5
[0, 7, 4, 9]
[10.0, 27.0, 53.0, 125.0]
0.0
115.0
[0, 3, 2, 9]
[10.5, 47.0, 54.0, 131.0]
0.0
120.5
[0, 1, 9]
[11.0, 60.0, 137.0]
0.0
126.0
[0, 9]
[11.5, 143.0]
0.0
131.5
[0, 8, 7, 2, 9]
[12.0, 16.0, 33.

[0, 8, 6, 2, 9]
[9.3, 27.2, 34.2, 65.6, 190.6]
0.0
181.29999999999998
[0, 4, 9]
[9.6, 58.6, 196.6]
0.0
187.0
[0, 9]
[9.9, 202.6]
0.0
192.7
[0, 4, 2, 9]
[10.2, 64.6, 71.6, 208.6]
0.0
198.4
[0, 5, 9]
[10.5, 49.6, 214.6]
0.0
204.1
[0, 8, 5, 9]
[10.8, 48.6, 55.6, 220.6]
0.0
209.79999999999998
[0, 2, 9]
[11.1, 77.6, 226.6]
0.0
215.5
[0, 7, 6, 9]
[11.4, 38.6, 45.6, 232.6]
0.0
221.2
[0, 8, 9]
[11.7, 54.6, 238.6]
0.0
226.9
[0, 9]
[12.0, 244.6]
0.0
232.6
[0, 9]
[12.3, 250.6]
0.0
238.29999999999998
[0, 8, 2, 9]
[12.6, 60.6, 83.6, 256.6]
0.0
244.00000000000003
[0, 7, 5, 3, 2, 1, 9]
[12.9, 44.6, 61.6, 68.6, 89.6, 96.6, 262.6]
0.0
249.70000000000002
[0, 9]
[13.2, 268.6]
0.0
255.40000000000003
[0, 7, 1, 9]
[13.5, 50.6, 102.6, 274.6]
0.0
261.1
[0, 7, 4, 9]
[13.8, 56.6, 70.6, 280.6]
0.0
266.8
[0, 4, 9]
[14.1, 76.6, 286.6]
0.0
272.5
[0, 2, 9]
[14.4, 95.6, 292.6]
0.0
278.20000000000005
[0, 5, 9]
[14.7, 67.6, 298.6]
0.0
283.90000000000003
[0, 6, 1, 9]
[15.0, 51.6, 108.6, 304.6]
0.0
289.6
itr 2
50
[2.5, a

[0, 8, 1, 9]
[1.0, 8.8, 16.8, 34.4]
0.0
33.4
[0, 4, 9]
[1.2, 2.2, 40.4]
0.0
39.199999999999996
[0, 9]
[1.4, 46.4]
0.0
45.0
[0, 7, 4, 3, 1, 9]
[1.6, 21.4, 28.4, 35.4, 42.4, 52.4]
0.0
50.8
[0, 6, 4, 9]
[1.8, 14.4, 34.4, 58.4]
0.0
56.6
[0, 5, 2, 9]
[2.0, 7.4, 14.4, 64.4]
0.0
62.400000000000006
[0, 5, 3, 9]
[2.2, 13.4, 41.4, 70.4]
0.0
68.2
[0, 5, 3, 1, 9]
[2.4, 19.4, 47.4, 54.4, 76.4]
0.0
74.0
[0, 9]
[2.6, 82.4]
0.0
79.80000000000001
[0, 9]
[2.8, 88.4]
0.0
85.60000000000001
[0, 9]
[3.0, 94.4]
0.0
91.4
[0, 4, 9]
[3.2, 40.4, 100.4]
0.0
97.2
[0, 5, 4, 9]
[3.4, 25.4, 46.4, 106.4]
0.0
103.0
[0, 7, 5, 1, 9]
[3.6, 27.4, 34.4, 60.4, 112.4]
0.0
108.80000000000001
[0, 5, 4, 9]
[3.8, 40.4, 52.4, 118.4]
0.0
114.60000000000001
[0, 7, 4, 9]
[4.0, 33.4, 58.4, 124.4]
0.0
120.4
[0, 3, 2, 9]
[4.2, 53.4, 60.4, 130.4]
0.0
126.2
[0, 1, 9]
[4.4, 66.4, 136.4]
0.0
132.0
[0, 9]
[4.6, 142.4]
0.0
137.8
[0, 8, 7, 2, 9]
[4.8, 14.8, 39.4, 66.4, 148.4]
0.0
143.6
[0, 9]
[5.0, 154.4]
0.0
149.4
[0, 1, 9]
[5.2, 72.4, 160.4]

[0, 5, 9]
[2.9, 46.2, 178.2]
0.0
175.29999999999998
[0, 8, 1, 9]
[3.0, 20.4, 78.2, 184.2]
0.0
181.2
[0, 8, 6, 2, 9]
[3.1, 26.4, 33.4, 72.2, 190.2]
0.0
187.1
[0, 4, 9]
[3.2, 64.2, 196.2]
0.0
193.0
[0, 9]
[3.3, 202.2]
0.0
198.89999999999998
[0, 4, 2, 9]
[3.4, 70.2, 78.2, 208.2]
0.0
204.79999999999998
[0, 5, 9]
[3.5, 52.2, 214.2]
0.0
210.7
[0, 8, 5, 9]
[3.6, 51.2, 58.2, 220.2]
0.0
216.6
[0, 2, 9]
[3.7, 84.2, 226.2]
0.0
222.5
[0, 7, 6, 9]
[3.8, 45.2, 52.2, 232.2]
0.0
228.39999999999998
[0, 8, 9]
[3.9, 57.2, 238.2]
0.0
234.29999999999998
[0, 9]
[4.0, 244.2]
0.0
240.2
[0, 9]
[4.1, 250.2]
0.0
246.1
[0, 8, 2, 9]
[4.2, 63.2, 90.2, 256.2]
0.0
252.0
[0, 7, 5, 3, 2, 1, 9]
[4.3, 51.2, 64.2, 71.2, 96.2, 103.2, 262.2]
0.0
257.9
[0, 9]
[4.4, 268.2]
0.0
263.8
[0, 7, 1, 9]
[4.5, 57.2, 109.2, 274.2]
0.0
269.7
[0, 7, 4, 9]
[4.6, 63.2, 76.2, 280.2]
0.0
275.59999999999997
[0, 4, 9]
[4.7, 82.2, 286.2]
0.0
281.5
[0, 2, 9]
[4.8, 102.2, 292.2]
0.0
287.4
[0, 5, 9]
[4.9, 70.2, 298.2]
0.0
293.3
[0, 6, 1, 9]
[5.0, 

[0, 8, 1, 9]
[0.5, 8.4, 16.4, 34.2]
0.0
33.7
[0, 4, 9]
[0.6, 1.6, 40.2]
0.0
39.6
[0, 9]
[0.7, 46.2]
0.0
45.5
[0, 7, 4, 3, 1, 9]
[0.8, 21.2, 28.2, 35.2, 42.2, 52.2]
0.0
51.400000000000006
[0, 6, 4, 9]
[0.9, 14.2, 34.2, 58.2]
0.0
57.300000000000004
[0, 5, 2, 9]
[1.0, 7.2, 14.2, 64.2]
0.0
63.2
[0, 5, 3, 9]
[1.1, 13.2, 41.2, 70.2]
0.0
69.10000000000001
[0, 5, 3, 1, 9]
[1.2, 19.2, 47.2, 54.2, 76.2]
0.0
75.0
[0, 9]
[1.3, 82.2]
0.0
80.9
[0, 9]
[1.4, 88.2]
0.0
86.8
[0, 9]
[1.5, 94.2]
0.0
92.7
[0, 4, 9]
[1.6, 40.2, 100.2]
0.0
98.60000000000001
[0, 5, 4, 9]
[1.7, 25.2, 46.2, 106.2]
0.0
104.5
[0, 7, 5, 1, 9]
[1.8, 27.2, 34.2, 60.2, 112.2]
0.0
110.4
[0, 5, 4, 9]
[1.9, 40.2, 52.2, 118.2]
0.0
116.3
[0, 7, 4, 9]
[2.0, 33.2, 58.2, 124.2]
0.0
122.2
[0, 3, 2, 9]
[2.1, 53.2, 60.2, 130.2]
0.0
128.1
[0, 1, 9]
[2.2, 66.2, 136.2]
0.0
134.0
[0, 9]
[2.3, 142.2]
0.0
139.89999999999998
[0, 8, 7, 2, 9]
[2.4, 14.4, 39.2, 66.2, 148.2]
0.0
145.79999999999998
[0, 9]
[2.5, 154.2]
0.0
151.7
[0, 1, 9]
[2.6, 72.2, 160.2]

...

[0, 8, 1, 9]
[0.5, 8.4, 16.4, 34.2]
0.0
33.7
[0, 4, 9]
[0.6, 1.6, 40.2]
0.0
39.6
[0, 9]
[0.7, 46.2]
0.0
45.5
[0, 7, 4, 3, 1, 9]
[0.8, 21.2, 28.2, 35.2, 42.2, 52.2]
0.0
51.400000000000006
[0, 6, 4, 9]
[0.9, 14.2, 34.2, 58.2]
0.0
57.300000000000004
[0, 5, 2, 9]
[1.0, 7.2, 14.2, 64.2]
0.0
63.2
[0, 5, 3, 9]
[1.1, 13.2, 41.2, 70.2]
0.0
69.10000000000001
[0, 5, 3, 1, 9]
[1.2, 19.2, 47.2, 54.2, 76.2]
0.0
75.0
[0, 9]
[1.3, 82.2]
0.0
80.9
[0, 9]
[1.4, 88.2]
0.0
86.8
[0, 9]
[1.5, 94.2]
0.0
92.7
[0, 4, 9]
[1.6, 40.2, 100.2]
0.0
98.60000000000001
[0, 5, 4, 9]
[1.7, 25.2, 46.2, 106.2]
0.0
104.5
[0, 7, 5, 1, 9]
[1.8, 27.2, 34.2, 60.2, 112.2]
0.0
110.4
[0, 5, 4, 9]
[1.9, 40.2, 52.2, 118.2]
0.0
116.3
[0, 7, 4, 9]
[2.0, 33.2, 58.2, 124.2]
0.0
122.2
[0, 3, 2, 9]
[2.1, 53.2, 60.2, 130.2]
0.0
128.1
[0, 1, 9]
[2.2, 66.2, 136.2]
0.0
134.0
[0, 9]
[2.3, 142.2]
0.0
139.89999999999998
[0, 8, 7, 2, 9]
[2.4, 14.4, 39.2, 66.2, 148.2]
0.0
145.79999999999998
[0, 9]
[2.5, 154.2]
0.0
151.7
[0, 1, 9]
[2.6, 72.2, 160.2]

[0, 8, 1, 9]
[3.0, 20.4, 78.2, 184.2]
0.0
181.2
[0, 8, 6, 2, 9]
[3.1, 26.4, 33.4, 72.2, 190.2]
0.0
187.1
[0, 4, 9]
[3.2, 64.2, 196.2]
0.0
193.0
[0, 9]
[3.3, 202.2]
0.0
198.89999999999998
[0, 4, 2, 9]
[3.4, 70.2, 78.2, 208.2]
0.0
204.79999999999998
[0, 5, 9]
[3.5, 52.2, 214.2]
0.0
210.7
[0, 8, 5, 9]
[3.6, 51.2, 58.2, 220.2]
0.0
216.6
[0, 2, 9]
[3.7, 84.2, 226.2]
0.0
222.5
[0, 7, 6, 9]
[3.8, 45.2, 52.2, 232.2]
0.0
228.39999999999998
[0, 8, 9]
[3.9, 57.2, 238.2]
0.0
234.29999999999998
[0, 9]
[4.0, 244.2]
0.0
240.2
[0, 9]
[4.1, 250.2]
0.0
246.1
[0, 8, 2, 9]
[4.2, 63.2, 90.2, 256.2]
0.0
252.0
[0, 7, 5, 3, 2, 1, 9]
[4.3, 51.2, 64.2, 71.2, 96.2, 103.2, 262.2]
0.0
257.9
[0, 9]
[4.4, 268.2]
0.0
263.8
[0, 7, 1, 9]
[4.5, 57.2, 109.2, 274.2]
0.0
269.7
[0, 7, 4, 9]
[4.6, 63.2, 76.2, 280.2]
0.0
275.59999999999997
[0, 4, 9]
[4.7, 82.2, 286.2]
0.0
281.5
[0, 2, 9]
[4.8, 102.2, 292.2]
0.0
287.4
[0, 5, 9]
[4.9, 70.2, 298.2]
0.0
293.3
[0, 6, 1, 9]
[5.0, 58.2, 115.2, 304.2]
0.0
299.2
itr 1
50
[11.666666666

[0, 5, 6, 7, 9]
[0.2, 1.2, 8.2, 15.2, 22.2]
0.0
22.0
[0, 1, 9]
[0.3, 1.3, 15.1]
0.0
14.799999999999999
[0, 8, 9]
[0.4, 2.4, 28.2]
0.0
27.8
[0, 8, 1, 9]
[0.5, 8.4, 16.4, 34.2]
0.0
33.7
[0, 4, 9]
[0.6, 1.6, 40.2]
0.0
39.6
[0, 9]
[0.7, 46.2]
0.0
45.5
[0, 7, 4, 3, 1, 9]
[0.8, 21.2, 28.2, 35.2, 42.2, 52.2]
0.0
51.400000000000006
[0, 6, 4, 9]
[0.9, 14.2, 34.2, 58.2]
0.0
57.300000000000004
[0, 5, 2, 9]
[1.0, 7.2, 14.2, 64.2]
0.0
63.2
[0, 5, 3, 9]
[1.1, 13.2, 41.2, 70.2]
0.0
69.10000000000001
[0, 5, 3, 1, 9]
[1.2, 19.2, 47.2, 54.2, 76.2]
0.0
75.0
[0, 9]
[1.3, 82.2]
0.0
80.9
[0, 9]
[1.4, 88.2]
0.0
86.8
[0, 9]
[1.5, 94.2]
0.0
92.7
[0, 4, 9]
[1.6, 40.2, 100.2]
0.0
98.60000000000001
[0, 5, 4, 9]
[1.7, 25.2, 46.2, 106.2]
0.0
104.5
[0, 7, 5, 1, 9]
[1.8, 27.2, 34.2, 60.2, 112.2]
0.0
110.4
[0, 5, 4, 9]
[1.9, 40.2, 52.2, 118.2]
0.0
116.3
[0, 7, 4, 9]
[2.0, 33.2, 58.2, 124.2]
0.0
122.2
[0, 3, 2, 9]
[2.1, 53.2, 60.2, 130.2]
0.0
128.1
[0, 1, 9]
[2.2, 66.2, 136.2]
0.0
134.0
[0, 9]
[2.3, 142.2]
0.0
139.8999

[0, 8, 5, 9]
[3.6, 51.2, 58.2, 220.2]
0.0
216.6
[0, 2, 9]
[3.7, 84.2, 226.2]
0.0
222.5
[0, 7, 6, 9]
[3.8, 45.2, 52.2, 232.2]
0.0
228.39999999999998
[0, 8, 9]
[3.9, 57.2, 238.2]
0.0
234.29999999999998
[0, 9]
[4.0, 244.2]
0.0
240.2
[0, 9]
[4.1, 250.2]
0.0
246.1
[0, 8, 2, 9]
[4.2, 63.2, 90.2, 256.2]
0.0
252.0
[0, 7, 5, 3, 2, 1, 9]
[4.3, 51.2, 64.2, 71.2, 96.2, 103.2, 262.2]
0.0
257.9
[0, 9]
[4.4, 268.2]
0.0
263.8
[0, 7, 1, 9]
[4.5, 57.2, 109.2, 274.2]
0.0
269.7
[0, 7, 4, 9]
[4.6, 63.2, 76.2, 280.2]
0.0
275.59999999999997
[0, 4, 9]
[4.7, 82.2, 286.2]
0.0
281.5
[0, 2, 9]
[4.8, 102.2, 292.2]
0.0
287.4
[0, 5, 9]
[4.9, 70.2, 298.2]
0.0
293.3
[0, 6, 1, 9]
[5.0, 58.2, 115.2, 304.2]
0.0
299.2
itr 2
50
[14.166666666666666, array([0.49030535]), 2, 50, 0, 0.0161784029006958, 0.012918420731309605]
[0, 7, 9]
[0.1, 2.1, 9.1]
0.0
9.0
[0, 5, 6, 7, 9]
[0.2, 1.2, 8.2, 15.2, 22.2]
0.0
22.0
[0, 1, 9]
[0.3, 1.3, 15.1]
0.0
14.799999999999999
[0, 8, 9]
[0.4, 2.4, 28.2]
0.0
27.8
[0, 8, 1, 9]
[0.5, 8.4, 16.4, 34.

[0, 5, 3, 1, 9]
[1.2, 19.2, 47.2, 54.2, 76.2]
0.0
75.0
[0, 9]
[1.3, 82.2]
0.0
80.9
[0, 9]
[1.4, 88.2]
0.0
86.8
[0, 9]
[1.5, 94.2]
0.0
92.7
[0, 4, 9]
[1.6, 40.2, 100.2]
0.0
98.60000000000001
[0, 5, 4, 9]
[1.7, 25.2, 46.2, 106.2]
0.0
104.5
[0, 7, 5, 1, 9]
[1.8, 27.2, 34.2, 60.2, 112.2]
0.0
110.4
[0, 5, 4, 9]
[1.9, 40.2, 52.2, 118.2]
0.0
116.3
[0, 7, 4, 9]
[2.0, 33.2, 58.2, 124.2]
0.0
122.2
[0, 3, 2, 9]
[2.1, 53.2, 60.2, 130.2]
0.0
128.1
[0, 1, 9]
[2.2, 66.2, 136.2]
0.0
134.0
[0, 9]
[2.3, 142.2]
0.0
139.89999999999998
[0, 8, 7, 2, 9]
[2.4, 14.4, 39.2, 66.2, 148.2]
0.0
145.79999999999998
[0, 9]
[2.5, 154.2]
0.0
151.7
[0, 1, 9]
[2.6, 72.2, 160.2]
0.0
157.6
[0, 9]
[2.7, 166.2]
0.0
163.5
[0, 3, 9]
[2.8, 59.2, 172.2]
0.0
169.39999999999998
[0, 5, 9]
[2.9, 46.2, 178.2]
0.0
175.29999999999998
[0, 8, 1, 9]
[3.0, 20.4, 78.2, 184.2]
0.0
181.2
[0, 8, 6, 2, 9]
[3.1, 26.4, 33.4, 72.2, 190.2]
0.0
187.1
[0, 4, 9]
[3.2, 64.2, 196.2]
0.0
193.0
[0, 9]
[3.3, 202.2]
0.0
198.89999999999998
[0, 4, 2, 9]
[3.4, 

In [195]:
Out.to_csv(r'C:\Users\dell\Desktop\IIT Slides\sem7\BTP\result3_TW_fin.csv',index=True)